In [ ]:
!pip install yfinance
!pip install pytickersymbols

     |████████████████████████████████| 5.5MB 9.5MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=d230920b15ebedc8d2673f03deefc5e654e28bed11ff7274e05b20c242fd2055
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 81kB 5.1MB/s 
     |████████████████████████████████| 276kB 7.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=1c7bd0dace539d649167a0073a6918f46238694d353c44d9d3ae8bd911fcd19a
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import yfinance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pytickersymbols import PyTickerSymbols
from google.colab import files

stock_data = PyTickerSymbols()
nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index()

horarios = ['09:30:00', '10:00:00', '10:30:00', '11:00:00'] #PARA INTERVALO DE 30 MIN (MAXIMO LIMITE DE 10 CANDLES)

# horarios = ['09:30:00','09:35:00', '09:40:00','09:45:00','09:50:00','09:55:00', '10:00:00', '10:05:00','10:10:00','10:15:00', '10:20:00', '10:25:00',
#             '10:30:00','10:35:00', '10:40:00','10:45:00','10:50:00','10:55:00', '11:00:00', '11:05:00','11:10:00','11:15:00', '11:20:00', '11:25:00',
#             '11:30:00','11:35:00', '11:40:00','11:45:00','11:50:00','11:55:00', '12:00:00', '12:05:00','12:10:00','12:15:00', '12:20:00', '12:25:00',
#             '12:30:00','12:35:00', '12:40:00','12:45:00','12:50:00','12:55:00', '13:00:00', '13:05:00','13:10:00','13:15:00', '13:20:00', '13:25:00',]

period = [10]   #días (para Bollinger)
#period= [5,10,15,20,5,10,15,20]
multiplier = [2.8]   #Desviaciones estándar (para Bollingers)
#multiplier = [2,2,2,2,2.5,2.5,2.5,2.5]
limit = [10]   #Limit of days to hold stock
#limit = [5,10,15,5,10,15,5,10,15,5,10,15]

cash = 5000
comision = 3

In [ ]:
df_totales = pd.DataFrame(['Total Trades', 'GREENS', 'PL(G)', 'PL/Trade(G)', 'Max(G)', 
                           'REDS', 'PL(R)','PL/Trade(R)', 'Max(R)', 'Total PL'], 
                          index=['Total Trades', 'GREENS', 'PL(G)', 'PL/Trade(G)', 'Max(G)', 
                           'REDS', 'PL(R)','PL/Trade(R)', 'Max(R)', 'Total PL'])
df_totales2 = df_totales.drop(columns=0)

In [ ]:
iteracion = pd.DataFrame(['Periodo (bb)', 'Desv Estandar', 'Límite días'], index=['Periodo (bb)', 'Std', 'Límite días']).drop(columns=0)

In [ ]:
totales = pd.DataFrame(['Total de trades:', 'Total de PL:', 'PL total/trade:', 'Porcentaje de Win Primer día:',
                        'Porcentaje de Win otros días:', 'Porcentaje de losers:','PL/Green trade:','PL/Red trade:'], 
                       index= ['Total de trades:', 'Total de PL:', 'PL total/trade:', 'Porcentaje de Win Primer día:',
                        'Porcentaje de Win otros días:', 'Porcentaje de losers:','PL/Green trade:','PL/Red trade:']).drop(columns=0)

In [ ]:
df = yfinance.download('QQQ',period='60d', interval='15m')

[*********************100%***********************]  1 of 1 completed


In [ ]:
#%%time

for k in range(len(period)):
  df_totales = pd.DataFrame(['Total Trades', 'GREENS', 'PL(G)', 'PL/Trade(G)', 'Max(G)', 
                           'REDS', 'PL(R)','PL/Trade(R)', 'Max(R)', 'Total PL'], 
                          index=['Total Trades', 'GREENS', 'PL(G)', 'PL/Trade(G)', 'Max(G)', 
                           'REDS', 'PL(R)','PL/Trade(R)', 'Max(R)', 'Total PL'])
  df_totales2 = pd.DataFrame.copy(df_totales.drop(columns=0))
  d = pd.DataFrame.copy(pd.DataFrame(index=df.index))
  d_cierres = pd.DataFrame.copy(pd.DataFrame(index=range(1, limit[k]+1)))

  for j in range(len(nasdaq_100.symbol)):

    df = yfinance.download(nasdaq_100.symbol[j],period='60d', interval='30m')#

    #df['UpperBand'] = df['Close'].rolling(period[k]).mean() + df['Close'].rolling(period[k]).std() * multiplier[k]
    df['LowerBand'] = df['Close'].rolling(period[k]).mean() - df['Close'].rolling(period[k]).std() * multiplier[k]

    df2 = df[['Open', 'Close', 'LowerBand']]

    df2['Outside'] = np.nan
    for i in range(len(df2)):
      if df2['Close'][i] < df2['LowerBand'][i]:
        df2['Outside'][i] = True
      else:
        df2['Outside'][i] = False

    df2['Trigger'] = np.nan
    for i in range(1, len(df2)):
      #CONDICIONES DE TRIGGER: 1) FUERA DEL BOLLINGER EL DIA PREVIO Y 2) EL DIA ACTUAL, Y 3) SE ENCUENTRA EN LOS HORARIOS PARA ALCANZAR DAY TRADING
      if df2['Outside'][i-1] == True and df2['Open'][i] < df2['LowerBand'][i-1] and df2.index[i].strftime('%H:%M:%S') in horarios:
        df2['Trigger'][i] = 1.0
  
    #df2['After Day'] = np.nan
    df2['PL'] = np.nan

    pl = 0
    closed = 0
    pl_list = []
    closed_list = []
    for i in range(1, len(df2)):
      if (df2['Trigger'][i] == 1.0 or pl < 0):
        pl = pl + (((df2['Close'][i] - df2['Open'][i])/df2['Open'][i]) * cash)
        closed = closed + 1
        #CONDICIONES PARA CERRAR POSICION:.................
        if pl > 0 or closed == limit[k] or df2.index[i] == df2.index[-1]:
          df2['PL'][i] = pl - comision
          pl_list.append(pl - comision)
          closed_list.append(closed)
          closed = 0
          pl = 0

    df2['Green'] = np.nan
    df2['Red'] = np.nan
    for i in range(1, len(df2)):
      if df2['PL'][i] > 0:
        df2['Green'][i] = df2['PL'][i]
      elif df2['PL'][i] < 0:
        df2['Red'][i] = df2['PL'][i]
  
    df_totales2.insert(j, nasdaq_100.symbol[j], [df2['PL'].count(), df2['Green'].count(), df2['Green'].sum(),
                      df2['Green'].mean(), df2['Green'].max(), 
                      df2['Red'].count(), df2['Red'].sum(), df2['Red'].mean(), 
                      df2['Red'].max(), df2['PL'].sum()])
  
    pl_list = df2['PL'].to_list()

    while len(pl_list) < len(d):
      pl_list.append(0)
      closed_list.append(0)
  
    dfclosed = pd.DataFrame(closed_list) #Se pasa la lista a df

    if len(closed_list) > 0:
      inter = pd.DataFrame(dfclosed[0].value_counts()) #Se sacan el número de ocurrencias
      inter = inter.sort_index().reindex(range(1,limit[k]+1), fill_value=0) #Se ordena y se llenan los espacios
      inter = inter[0].to_list()
      #d.insert(j, nasdaq_100.symbol[j], pl_list)
    d_cierres.insert(j, nasdaq_100.symbol[j], inter)
  
  d_cierres['sum'] = d_cierres.sum(axis=1)
  iteracion.insert(k,k, [period[k], multiplier[k], limit[k]])
  df_totales3 = df_totales2.transpose()
  totales.insert(k, k, [df_totales3['Total Trades'].sum(), df_totales3['Total PL'].sum(), df_totales3['Total PL'].sum()/df_totales3['Total Trades'].sum(),
                 d_cierres['sum'][1]/df_totales3['Total Trades'].sum(), (df_totales3['GREENS'].sum()-d_cierres['sum'][1])/df_totales3['Total Trades'].sum(),
                 df_totales3['REDS'].sum()/df_totales3['Total Trades'].sum(), df_totales3['PL/Trade(G)'].mean(), df_totales3['PL/Trade(R)'].mean()])

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

[*********************100%***********************]  1 of 1 completed


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [ ]:
df2

,Open,Close,LowerBand,Outside,Trigger,PL,Green,Red
Datetime,,,,,,,,
2020-11-23 09:30:00-05:00,432.010010,420.299988,NaN,0.0,NaN,NaN,NaN,NaN
2020-11-23 10:00:00-05:00,420.709503,420.450012,NaN,0.0,NaN,NaN,NaN,NaN
2020-11-23 10:30:00-05:00,420.345001,425.115295,NaN,0.0,NaN,NaN,NaN,NaN
2020-11-23 11:00:00-05:00,425.109985,424.850006,NaN,0.0,NaN,NaN,NaN,NaN
2020-11-23 11:30:00-05:00,424.900391,430.979095,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2021-02-19 13:30:00-05:00,417.880005,418.380005,414.497673,0.0,NaN,NaN,NaN,NaN
2021-02-19 14:00:00-05:00,418.500000,416.839996,414.261510,0.0,NaN,NaN,NaN,NaN
2021-02-19 14:30:00-05:00,417.000000,416.940002,413.643051,0.0,NaN,NaN,NaN,NaN


In [ ]:
df_totales2['sum'] = df_totales2.sum(axis=1)
df_totales2

,AAPL,CSCO,INTC,MSFT,WBA,ATVI,ADBE,ALXN,GOOGL,AMZN,AMGN,ADI,AMAT,ADSK,ADP,BIDU,BIIB,CERN,CHTR,CHKP,CTSH,CMCSA,COST,CSX,DLTR,EBAY,EA,FB,FAST,FISV,GILD,ILMN,INCY,INTU,ISRG,JD,KHC,LRCX,MAR,MXIM,...,TXN,VRSK,VRTX,XLNX,BKNG,EXC,AMD,ALGN,AEP,ANSS,AVGO,CDNS,CTAS,CPRT,FOXA,IDXX,KLAC,SNPS,VRSN,ASML,CDW,DXCM,PEP,XEL,TEAM,DOCU,KDP,LULU,MRVL,MTCH,MELI,MRNA,OKTA,PTON,PDD,SGEN,SPLK,WDAY,ZM,sum
Total Trades,0.0,1.000000,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0,1.000000,0.0,1.000000,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,1.000000,0.0,0.0,...,0.0,0.0,0.0,1.000000,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,2.000000,0.0,1.000000,0.0,0.0,23.000000
GREENS,0.0,1.000000,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0,1.000000,0.0,1.000000,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,1.000000,0.0,0.0,...,0.0,0.0,0.0,1.000000,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,1.000000,0.0,1.000000,0.0,0.0,20.000000
PL(G),0.0,8.688153,0.0,0.0,0.0,76.356355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.472354,0.0,80.368734,0.0,0.0,0.0,0.0,19.503183,0.0,3.078079,0.0,0.0,0.0,3.244766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,50.110541,0.0,9.880669,0.0,0.0,...,0.0,0.0,0.0,83.952962,1.783617,0.0,41.422574,0.0,0.0,0.0,0.0,40.650396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.429871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.947998,0.0,77.988474,0.0,88.572261,0.0,0.0,809.616978
PL/Trade(G),NaN,8.688153,NaN,NaN,NaN,76.356355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.472354,NaN,80.368734,NaN,NaN,NaN,NaN,19.503183,NaN,3.078079,NaN,NaN,NaN,3.244766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.110541,NaN,9.880669,NaN,NaN,...,NaN,NaN,NaN,83.952962,1.783617,NaN,41.422574,NaN,NaN,NaN,NaN,40.650396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.429871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.947998,NaN,77.988474,NaN,88.572261,NaN,NaN,809.616978
Max(G),NaN,8.688153,NaN,NaN,NaN,76.356355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.472354,NaN,80.368734,NaN,NaN,NaN,NaN,19.503183,NaN,3.078079,NaN,NaN,NaN,3.244766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.110541,NaN,9.880669,NaN,NaN,...,NaN,NaN,NaN,83.952962,1.783617,NaN,41.422574,NaN,NaN,NaN,NaN,40.650396,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.429871,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.947998,NaN,77.988474,NaN,88.572261,NaN,NaN,809.616978
REDS,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.000000,0.0,0.0,3.000000
PL(R),0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,-90.365863,0.0,0.000000,0.0,0.0,-93.241245
PL/Trade(R),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-90.365863,NaN,NaN,NaN,NaN,-93.241245
Max(R),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
#df_totales2.loc['Max(G)'].sort_values().dropna().tail(6).sum()

In [ ]:
df2.loc[(df2['Outside']==1) | (df2['Trigger']==1) | (df2['PL']<0) | (df2['PL']>0)]

,Open,Close,LowerBand,Outside,Trigger,PL,Green,Red
Datetime,,,,,,,,


In [ ]:
iteracion

,0
Periodo (bb),10.0
Std,2.8
Límite días,10.0


In [ ]:
totales

,0
Total de trades:,23.000000
Total de PL:,716.375732
PL total/trade:,31.146771
Porcentaje de Win Primer día:,0.608696
Porcentaje de Win otros días:,0.260870
Porcentaje de losers:,0.130435
PL/Green trade:,40.480849
PL/Red trade:,-31.080415


In [ ]:
# print('Total de trades:', df_totales3['Total Trades'].sum())
# print('Total de PL:',df_totales3['Total PL'].sum())
# print('PL total/trade:',df_totales3['Total PL'].sum()/df_totales3['Total Trades'].sum())
# print('-'*50)
# print('Porcentaje de Win Primer día:',d_cierres['sum'][1]/df_totales3['Total Trades'].sum())
# print('Porcentaje de Win otros días:',(df_totales3['GREENS'].sum()-d_cierres['sum'][1])/df_totales3['Total Trades'].sum())
# print('Porcentaje de losers:',df_totales3['REDS'].sum()/df_totales3['Total Trades'].sum())
# print('-'*50)
# print('PL/Green trade:',df_totales3['PL/Trade(G)'].mean())
# print('PL/Red trade:',df_totales3['PL/Trade(R)'].mean())

In [ ]:
#DATAFRAME DE LOS DIAS QUE TOMÓ UNA POSICION PARA CERRAR, POR ACCIÓN
print(d_cierres['sum'].sum())
d_cierres

23


,AAPL,CSCO,INTC,MSFT,WBA,ATVI,ADBE,ALXN,GOOGL,AMZN,AMGN,ADI,AMAT,ADSK,ADP,BIDU,BIIB,CERN,CHTR,CHKP,CTSH,CMCSA,COST,CSX,DLTR,EBAY,EA,FB,FAST,FISV,GILD,ILMN,INCY,INTU,ISRG,JD,KHC,LRCX,MAR,MXIM,...,TXN,VRSK,VRTX,XLNX,BKNG,EXC,AMD,ALGN,AEP,ANSS,AVGO,CDNS,CTAS,CPRT,FOXA,IDXX,KLAC,SNPS,VRSN,ASML,CDW,DXCM,PEP,XEL,TEAM,DOCU,KDP,LULU,MRVL,MTCH,MELI,MRNA,OKTA,PTON,PDD,SGEN,SPLK,WDAY,ZM,sum
1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,14
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


In [ ]:
#DATAFRAME CON EL P/L RESULTANTE AL CERRAR LA POSICIÓN, POR DÍA ANALIZADO Y POR ACCIÓN
#d['sum'] = d.sum(axis=1).cumsum()
#d

In [ ]:
#P/L ACUMULADO
# plt.figure(figsize=[24,8])
# d['sum'].plot()

In [ ]:
#ANÁLISIS FINAL POR ACCION
df_totales3 = df_totales2.transpose()
#df_totales3

In [ ]:
print('Total de trades:', df_totales3['Total Trades'].sum())
print('Total de PL:',df_totales3['Total PL'].sum())
print('PL total/trade:',df_totales3['Total PL'].sum()/df_totales3['Total Trades'].sum())
print('-'*50)
print('Porcentaje de Win Primer día:',d_cierres['sum'][1]/df_totales3['Total Trades'].sum())
print('Porcentaje de Win otros días:',(df_totales3['GREENS'].sum()-d_cierres['sum'][1])/df_totales3['Total Trades'].sum())
print('Porcentaje de losers:',df_totales3['REDS'].sum()/df_totales3['Total Trades'].sum())
print('-'*50)
print('PL/Green trade:',df_totales3['PL/Trade(G)'].mean())
print('PL/Red trade:',df_totales3['PL/Trade(R)'].mean())

Total de trades: 46.0
Total de PL: 1432.7514644590992
PL total/trade: 31.146770966502157
--------------------------------------------------
Porcentaje de Win Primer día: 0.30434782608695654
Porcentaje de Win otros días: 0.5652173913043478
Porcentaje de losers: 0.13043478260869565
--------------------------------------------------
PL/Green trade: 77.10637882605184
PL/Red trade: -46.620622721997385


In [ ]:
#VERSION PARA SHORT TRADE:::::::::::::::::::::::::::::: #se agrega condicion para que sean los mismos periodos en cada stock (intra y daily)
      elif gap < 0 and data.index[i].strftime('%H:%M:%S') == '09:30:00' and len(data) <= (len(periods)*60) and len(data_day) == 60:
        long_bar_percentage = (data['Low'][i] - data['High'][i])/data['High'][i]
        #CONDICIONES PARA QUE EL "IGNITING" BAR SEA CANDIDATO: 
        #2)... Que el movimiento sea menor o igual a -1% en la Igniting bar
        #3)... Que el Igniting sea barra roja (close abajo del open) 4)...Que el Close de la barra sea mas bajo que el Low del día previo (mas potencia)
        #5)...Low de la barra siguiente sea menor al Low del "Igniting" Bar 6)... Que el Low segundo bar esté por arriba de una cuarta parte del Igniting
        if (long_bar_percentage <= -porcentaje[p] and data['Close'][i] < data['Open'][i]
          and data['Close'][i] < data_day['Low'].loc[data.index[i-1].strftime('%Y-%m-%d')] and data['Low'][i]*(1+tol) <= data['Low'][i+1] 
          and ((data['High'][i] - (data['High'][i] - data['Low'][i])/2)) > data['High'][i+1]):
          data['Igniting Bar'][i] = long_bar_percentage
    
        entry = 0
        stop = 0
        target = 0
        #Loop para definir si se entra al trade, y se definen stops, entries y targets
        if data['Igniting Bar'][i] > 0:
          for k in range(i+2, i+len(periods)-1):
            if data['Low'][k] < data['Low'][i]:
              data['Trigger'][i] = True

              entry = data['Low'][i]
              data['Entry_Pr'][i] = entry
              stop = entry - data['High'][i+1]
              data['Stop_Pr'][i] = data['High'][i+1]
              target = entry - (stop*risk[p])
              data['Target_Pr'][i] = target
              break
    
        #Se define si el trade toca el stop, target o si termina en green o red al final del día
        pl = 0
        if data['Trigger'][i] == True:
          for j in range(k, i+len(periods)):
            if data['High'][j] > data['High'][i+1]:
              data['Stopped'][i] = -1
              trades_down += 1
              PL_total += -1
              break
            elif data['Low'][j] <= target:
              data['Target'][i] = risk[p]
              trades_down += 1
              PL_total += risk[p]
              break
            elif j == (i+len(periods)-1):
              pl = (entry - data['Close'][j]) / (data['High'][i+1] - entry)
              if pl < 0: 
                data['Red'][i] = pl
                trades_down += 1
                PL_total += pl
              else: 
                data['Green'][i] = pl
                trades_down += 1
                PL_total += pl